<img src="img/mmselfsup_logo.png">

# 自监督预训练模型的评估：“分类” 下游任务 之 SVM 评估

<a href="https://colab.research.google.com/github/wangruohui/OpenMMLabCourse/blob/main/codes/MMSelfSup_tutorials/【4】自监督预训练模型的评估：“分类”%20下游任务%20之%20SVM%20评估.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMSelfSup Repo**：[https://github.com/open-mmlab/mmselfsup](https://github.com/open-mmlab/mmselfsup)

**MMSelfSup 官方文档链接**：[https://mmselfsup.readthedocs.io/en/latest](https://mmselfsup.readthedocs.io/en/latest)

**作者**：OpenMMLab

## 0. 任务介绍

SVM 评估也是评估自监督预训练模型的常用基准之一。它的 **`基本流程`** 如下：
1. 每张图像送入自监督预训练好的模型提取特征
2. 用所有训练集图片的特征向量来训练支持向量机 SVM 
3. 将测试集图片送入训练好的 SVM，将其分类性能作为评估从自监督学习方法中学习图像特征质量的手段，与其他自监督模型进行比较

在使用 “分类” 下游任务对自监督预训练模型进行评估的方法中，MMSelfSup 目前实现了以下几种方法：(详细请参考官方[文档](https://mmselfsup.readthedocs.io/zh_CN/latest/tutorials/6_benchmarks.html#id2))
+ **VOC SVM / Low-shot SVM**
+ 线性评估
+ ImageNet 半监督分类
+ ImageNet 最邻近分类

## 1. 环境配置

### 1.1 查看 Python、PyTorch 和 Torchvision 的版本

In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!gcc --version

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### 1.2 安装 MMSelfSup 的依赖库：MMCV

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv

### 1.3  安装 MMSelfSup

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd /content/mmselfsup

In [ ]:
# Install MMSelfSup from source
!pip install -e . 

### 1.4 检查安装是否正确

In [ ]:
import mmselfsup
print(mmselfsup.__version__)

## 2. 准备数据集

**注意**：目前，MMSelfSup 的 SVM 评估只支持了 `PASCAL VOC 2017` 数据集。

### 2.0 数据集介绍

本教程将在 `PASCAL VOC 2007` 数据集上对自监督预训练好的模型进行 SVM 评估。

该数据集包含有 20 个类别。训练集有 5011 张图片，测试集有 4952 张图片。

数据集官方下载地址：http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html

### 2.1 下载数据集

使用如下命令，自动将 PASCAL VOC 2007 下载到 `$YOUR_DATA_ROOT` 文件夹中。

```shell
bash tools/data_converters/prepare_voc07_cls.sh $YOUR_DATA_ROOT
```

In [ ]:
% cd /content/mmselfsup
!bash tools/data_converters/prepare_voc07_cls.sh data

In [ ]:
!rm -rf data/VOCtest_06-Nov-2007.tar
!rm -rf data/VOCtrainval_06-Nov-2007.tar

## 3. 准备自监督预训练模型

目前，MMSelfSup 里 SVM 评估的模型对象既可以是 `自监督预训练模型提取 backbone 部分的权重文件`，也可以是 `自监督预训练过程中直接保存下来的名为 epoch_*.pth 的 ckeckpoint 文件`。所以，我们这里需要准备好两种模型文件，方便后面对这两种文件进行 SVM 评估的代码演示。

### 3.1 准备自监督预训练模型提取 backbone 部分的权重文件

**注意：MMSelfSup 的 [模型库](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/model_zoo.md) 中的模型文件都已经提取过 backbone 权值，不需要再次提取！**我们直接使用即可。

在模型库中找到在 SimCLR 的预训练模型文件 `simclr_resnet50_8xb32-coslr-200e_in1k`，下载放在 `checkpoints` 文件夹里 

In [ ]:
%cd /content/mmselfsup
!mkdir checkpoints
!wget https://download.openmmlab.com/mmselfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth -P checkpoints

### 3.2 准备自监督预训练过程中直接保存下来的 ckeckpoint 文件

我们使用第一个教程 `模型自监督预训练 之 SimCLR` 中训练保存下来的 `epoch_1.pth` 文件进行演示，该文件可以从 [这里](https://download.openmmlab.com/mmselfsup/tutorial/epoch_1.pth) 下载，存放在文件夹 `mmselfsup/work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200` 里。

In [ ]:
%cd /content/mmselfsup
!mkdir -p work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200
!wget https://download.openmmlab.com/mmselfsup/tutorial/epoch_1.pth  -P  work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200

同时，准备好第一个教程 `模型自监督预训练 之 SimCLR` 中的自监督预训练配置文件 `simclr_resnet50_1xb32-coslr-1e_tinyin200.py`。

In [ ]:
%cd /content/mmselfsup

In [ ]:
%%writefile /content/mmselfsup/configs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200.py

_base_ = 'simclr_resnet50_8xb32-coslr-200e_in1k.py'

# dataset
data = dict(
    samples_per_gpu=32, 
    workers_per_gpu=2,
    train=dict(
        data_source=dict(
            data_prefix='data/tiny-imagenet-200/train',
            ann_file='data/tiny-imagenet-200/train.txt',
        )
    )
)

# optimizer
optimizer = dict(
    lr=0.3 * ((32 * 1) / (32 * 8)),
)

runner = dict(max_epochs=1)

## 4. 自监督预训练模型的 SVM 评估

下面，我们分别演示对 `自监督预训练模型提取 backbone 部分的权重文件` 和 `自监督预训练过程中直接保存下来的名为 epoch_*.pth 的 ckeckpoint 文件` 两种文件进行 SVM 评估。

相关代码文件见：[tools/benchmarks/classification/svm_voc07](https://github.com/open-mmlab/mmselfsup/tree/master/tools/benchmarks/classification/svm_voc07)

### 4.1 对自监督预训练模型的 backbone 权重文件进行 SVM 评估

在运行自监督预训练模型的 SVM 评估时，我们使用 [dist_test_svm_pretrain.sh](https://github.com/open-mmlab/mmselfsup/blob/master/tools/benchmarks/classification/svm_voc07/dist_test_svm_pretrain.sh) 脚本来启动自监督预训练模型 backbone 权重文件的评估。

```shell
bash tools/benchmarks/classification/svm_voc07/dist_test_svm_pretrain.sh ${SELFSUP_CONFIG} ${GPUS} ${PRETRAIN} ${FEATURE_LIST}

```

参数：
- SELFSUP_CONFIG：自监督预训练的配置文件
- GPUS：使用 GPU 的数量
- PRETRAIN：自监督预训练模型的 backbone 权重文件所在的路径。
- FEATURE_LIST：该参数的值是一个字符串，用于指定评估从 layer1 到 layer5 的特征，默认值为 `"feat5"`，表示评估 layer5 的特征。如果想评估 layer3，那么该参数的值是 `"feat3"`；如果想评估所有特征，那么该参数的值是 `"feat1 feat2 feat3 feat4 feat5"`（注意用空格分隔）。

In [ ]:
%cd /content/mmselfsup

In [ ]:
!bash tools/benchmarks/classification/svm_voc07/dist_test_svm_pretrain.sh \
configs/selfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k.py \
1 \
checkpoints/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth \
"feat5"

### 4.2 对自监督预训练过程中直接保存下来的 ckeckpoint 文件进行 SVM 评估

在运行自监督预训练模型的 SVM 评估时，我们使用 [dist_test_svm_epoch.sh](https://github.com/open-mmlab/mmselfsup/blob/master/tools/benchmarks/classification/svm_voc07/dist_test_svm_epoch.sh) 脚本来启动自监督预训练过程中直接保存下来 ckeckpoint 文件的评估。

```shell
GPUS=${GPUS} bash tools/benchmarks/classification/svm_voc07/dist_test_svm_epoch.sh ${SELFSUP_CONFIG} ${EPOCH} ${FEATURE_LIST}
```

参数：
- GPUS：默认使用的 GPU 数量是 8，如果需要调整，需要加上该环境变量。
- SELFSUP_CONFIG：自监督预训练的配置文件
- EPOCH：想要测试 checkpoint 文件的 epoch 数。例如：该参数的值为 5，意味着测名为 `epoch_5.pth` 的 checkpoint 文件。
- FEATURE_LIST：该参数的值是一个字符串，用于指定评估从 layer1 到 layer5 的特征，默认值为 `"feat5"`，表示评估 layer5 的特征。如果想评估 layer3，那么该参数的值是 `"feat3"`；如果想评估所有特征，那么该参数的值是 `"feat1 feat2 feat3 feat4 feat5"`（注意用空格分隔）。

In [ ]:
%cd /content/mmselfsup

In [ ]:
!GPUS=1 bash tools/benchmarks/classification/svm_voc07/dist_test_svm_epoch.sh \
configs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200.py \
1 \
"feat5"